In [4]:
import sys
from pathlib import Path

# 프로젝트 루트 경로 추가
ROOT = Path.cwd().parent  # jupyter/ 에서 한 단계 위로
sys.path.insert(0, str(ROOT))

# 이제 import 가능
from scripts.es_query import get_es_client, get_index_name, cmd_stats, cmd_list_docs, cmd_get_doc, cmd_search
import sys
from pathlib import Path

ROOT = Path.cwd().parent
sys.path.insert(0, str(ROOT))

from elasticsearch import Elasticsearch
from backend.config.settings import search_settings

In [5]:
# ES 클라이언트 생성 (호스트에서 실행 시)
es = Elasticsearch(hosts=["http://localhost:8002"])
# 인덱스 이름
index = f"{search_settings.es_index_prefix}_{search_settings.es_env}_current"

# 예: 문서 목록 조회
result = es.search(
    index=index,
    body={
    "size": 0,
    "aggs": {
    "docs": {"terms": {"field": "doc_id.keyword", "size": 100}}
    }
    }
)

result["aggregations"]["docs"]["buckets"]

[{'key': 'set_up_manual_supra_vm', 'doc_count': 316},
 {'key': 'set_up_manual_ecolite_3000', 'doc_count': 284},
 {'key': 'set_up_manual_supra_np', 'doc_count': 266},
 {'key': 'set_up_manual_supra_n', 'doc_count': 265},
 {'key': 'set_up_manual_supra_nm', 'doc_count': 239},
 {'key': 'set_up_manual_supra_xq', 'doc_count': 190},
 {'key': 'set_up_manual_geneva_stp300_xp_r6', 'doc_count': 168},
 {'key': 'set_up_manual_integer_plus', 'doc_count': 167},
 {'key': 'set_up_manual_precia', 'doc_count': 159},
 {'key': 'set_up_manual_zedius_xp_supra_xp', 'doc_count': 145},
 {'key': 'global_sop_integer_plus_all_am_vacuum_line', 'doc_count': 135},
 {'key': 'global_sop_integer_plus_all_pm_vacuum_line', 'doc_count': 133},
 {'key': 'global_sop_precia_all_pm_chuck', 'doc_count': 132},
 {'key': 'set_up_manual_ecolite_ii_400', 'doc_count': 128},
 {'key': 'global_sop_integer_plus_all_efem_robot', 'doc_count': 126},
 {'key': 'global_sop_integer_plus_all_pm_pin_motor', 'doc_count': 126},
 {'key': 'global_sop_s

In [34]:
result["aggregations"]['docs']

{'doc_count_error_upper_bound': 0,
 'sum_other_doc_count': 6536,
 'buckets': [{'key': 'pe_preprocess_data_set_up_manual_set_up_manual_supra_vm',
   'doc_count': 316},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_ecolite_3000',
   'doc_count': 284},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_supra_np',
   'doc_count': 266},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_supra_n',
   'doc_count': 265},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_supra_nm',
   'doc_count': 239},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_supra_xq',
   'doc_count': 190},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_geneva_stp300_xp_r6',
   'doc_count': 168},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_integer_plus',
   'doc_count': 167},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_precia',
   'doc_count': 159},
  {'key': 'pe_preprocess_data_set_up_manual_set_up_manual_zedius_xp_supra_xp',
   'doc_co

In [4]:

# 2. 특정 doc_id의 모든 chunk 조회
doc_id = "pe_preprocess_data_sop_pdfs_global_sop_supra_xp_all_pm_pirani_gauge"
result = es.search(
    index=index,
    body={
    "query": {"term": {"doc_id.keyword": doc_id}},
    "size": 100,
    "sort": [{"page": "asc"}],
    "_source": {"excludes": ["embedding"]}
    }
)
for hit in result["hits"]["hits"]:
    print(hit["_source"]["content"][:200])



In [74]:
from openai import OpenAI
    
# vLLM 서버에 연결 
client = OpenAI(   
    base_url="http://localhost:8003/v1",   
    api_key="dummy",  # vLLM은 API key 불필요  
)  

def summarize(text: str) -> str:   
    response = client.chat.completions.create( 
    model="openai/gpt-oss-20b",    
    messages=[ 
    {"role": "system", "content": "주어진 텍스트를 간결하게 요약하세요."}, 
    {"role": "user", "content": text}  
    ], 
    temperature=0, 
    max_tokens=512,    
    )  
    return response.choices[0].message.content 
    



page_summaries = []    
for doc in result["hits"]["hits"]:  
    summary = summarize(doc['_source']['content'])  # Document 객체라면 .page_content
    doc['_source']['page']
    page_summaries.append(summary) 

    print(summary) 
    

**요약**  
이 문서는 ZEDIUS XP 설비의 PM(예방정비) Pirani 게이지 작업에 필요한 절차, 주의사항, 준비물 등을 정리한 Global SOP입니다.  
- **버전**: 1 (수정 기록 포함)  
- **페이지**: 1/40  
- **주요 내용**: 안전, 안전 라벨, 사고 사례, 환경 보호구 체크시트, 작업자 위치, 개정 이력, 작업 리스트(6페이지·22페이지)  
- **작업 리스트**: PM PIRANI GAUGE ADJ(6페이지), PM PIRANI GAUGE REP(22페이지)  
- **용지**: PS‑320‑R0 A4(210×297)  
이 SOP는 설비 정비 시 안전과 효율을 확보하기 위한 지침을 제공합니다.
**SOP 요약 – ZEDIUS XP_ALL_PM_PIRANI 게이지**

- **문서 정보**: Global SOP, Revision 1, 페이지 2/40  
- **주요 안전 지침**  
  1. **화상 주의** – PM(프로세스 모니터) 온도가 250 °C, 챔버 구조는 90 °C이므로 작업 시 화상 위험이 큼.  
  2. **챔버 비활성화 확인** – 자동 기능이 오작동하지 않도록 작업 전 챔버가 Disable 상태인지 반드시 점검.  
- **안전 라벨**  
  - **Burn Warning (Burn 경고)** 라벨이 부착되어 있음.  
  - 라벨은 PS‑320‑R0, A4(210 × 297 mm) 크기로 표시.  

이 SOP는 고온 환경에서의 화상 방지와 자동 기능 오류 방지를 위해 필요한 기본 안전 절차를 정리한 문서입니다.
**요약**

- **화상 정의**: 불·뜨거운 물·화학물질 등으로 피부·조직이 손상되는 현상. 1도~4도로 구분하며, 부위·넓이·깊이·연령을 고려해 중증도를 판단.
- **화상 위험 종류**  
  1. 열상화상 (고온)  
  2. 화학화상 (화학물질 접촉)  
  3. 전기화상 (전기 손상)  
  4. 흡입화상 (뜨거운 공기·연기 흡입)
- **예방 조치**  
  - 

In [75]:
doc['_source']['page']

40

'```markdown # Global SOP_ZEDIUS XP_ALL_PM_PIRANI GAUGE | Global SOP No: | | | --- | --- | | Revision No: | 1 | | Page: | 1/40 | ## Scope 이 Global SOP는 ZEDIUS XP 설비의 PM Pirani Gauge 작업 시에 필요한 작업 절차와 주의 사항, 준비 사항에 대해서 작성한 Global SOP 입니다. ## Contents 1. Safety 2. Safety Label 3. 사고 사례 4. 환경 안전 보호구 Check Sheet 5. Worker Location 6. Revision History 7. Work List 1) PM PIRANI GAUGE ADJ ----------------------------- 6 page 2) PM PIRANI GAUGE REP ----------------------------- 22 page PS-320-R0 A4(210x297) ```'

In [6]:
# 3. 텍스트 검색  
query = "manometer"
result = es.search(   
    index=index,
    body={    
    "query": {    
    "multi_match": {  
    "query": query,
    "fields": ["content", "search_text"]  
    } 
    },
    "size": 10,
    "_source": {"excludes": ["embedding"]}    
    } 
)
for hit in result["hits"]["hits"]:
    print(f"[{hit['_score']:.2f}] {hit['_source']['doc_id']}")
    print(hit["_source"]["content"][:200])
    print()
    

[9.78] global_sop_integer_plus_all_pm_manometer
# Global SOP_INTEGER Plus_REP_MANOMETER Global SOP No: Revision No: 1 Page: 10 / 20 ## 5. Flow Chart Start 1. Global SOP 및 안전 사항 Check 2. PM Disable 3. Manometer Pressure Check 4. Manometer Parameter 

[9.70] odl_out_supra_xp_sub_unit_trouble_shooting_guide_igs_block_abnormal_annotated
# Trouble Shooting Guide [IGS Block Abnormal] | Failure symptoms | Check point | Key point | |---|---|---| | Gas Exhaust Alarm | A-1. Manometer | - Adjust Damper<br>- Manometer Zero Calibration<br>- M

[9.68] global_sop_supra_xp_all_pm_manometer
```markdown # Global SOP_ZEDIUS XP_ALL_PM_MANOMETER REPLACEMENT Global SOP No : Revision No: 1 Page: 13/32 | Flow | Procedure | Tool & Point | | :--- | :--- | :--- | | 6. Manometer 교체 | ![](image1.png

[9.61] supra_xp_sub_unit_trouble_shooting_guide_igs_block_abnormal
```markdown # Trouble Shooting Guide [IGS Block Abnormal] Confidential II | Failure symptoms | Check point | Key point | | :--- | :--- | :--- | | Gas 

In [1]:

from elasticsearch import Elasticsearch
from backend.config.settings import search_settings

es = Elasticsearch(hosts=["http://localhost:8002"])
index = f"{search_settings.es_index_prefix}_{search_settings.es_env}_current"

# 모든 문서 삭제
resp = es.delete_by_query(
    index=index,
    body={"query": {"match_all": {}}},
    conflicts="proceed",
    refresh=True,
)

ModuleNotFoundError: No module named 'backend'

In [ ]:
python scripts/vlm_es_ingest.py --input "/home/llm-share/datasets/pe_agent_data/pe_preprocess_data/set_up_manual" --doc-type setup --lang ko --tenant-id 20251223 --workers 2

In [10]:
# Myservice 문서만 조회
result = es.search(
    index=index,
    body={
        "query": {"term": {"doc_type.keyword": "myservice"}},
        "size": 10,
        "_source": {"excludes": ["embedding"]}
    }
)
result['hits']['hits'][0]

{'_index': 'rag_chunks_dev_current',
 '_id': '40001648#0000',
 '_score': 2.14987,
 '_source': {'doc_id': '40001648',
  'chunk_id': '40001648#0000',
  'page': 0,
  'content': 'EPAH54 설비 정상화를 위한 Part 장착 요청 건',
  'search_text': 'status EPAH54 설비 정상화를 위한 Part 장착 요청 건 myservice batch',
  'lang': 'ko',
  'doc_type': 'myservice',
  'tenant_id': '',
  'project_id': '',
  'pipeline_version': 'v1',
  'content_hash': '31c3c9d9847c0b32',
  'created_at': '2025-12-30T15:58:48.000379+00:00',
  'updated_at': '2025-12-30T15:58:48.000379+00:00',
  'device_name': 'SUPRA Vplus',
  'doc_description': '설비 정상화를 위한 Part 장착 요청 건',
  'chapter': 'status',
  'chunk_summary': 'EPAH54 장비 정상화를 위한 부품 장착 요청이 접수되었습니다.',
  'chunk_keywords': ['EPAH54', 'normalization', 'part installation'],
  'tags': ['myservice', 'batch']}}